# 1. Configuración de Spark 

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

packages = [
    "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0",
    "org.apache.hadoop:hadoop-aws:3.3.4",
    "com.amazonaws:aws-java-sdk-bundle:1.12.262"
]

spark = SparkSession.builder \
    .appName("AlertasTemperaturaStreaming") \
    .config("spark.jars.packages", ",".join(packages)) \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")

In [5]:
from pyspark.sql.functions import from_json, col, to_timestamp
from pyspark.sql.types import *

# 1. Esquema para datos JSON
schema_temp = StructType([
    StructField("sensor_id", StringType(), True),
    StructField("temperature", DoubleType(), True),
    StructField("timestamp", StringType(), True),
    StructField("location", StringType(), True)
])

# 2. Lectura de Kafka
kafka_temp_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:29092") \
    .option("subscribe", "temperature-sensors") \
    .option("startingOffsets", "latest") \
    .load()

# 3. Parseo de JSON 
parsed_temp_df = kafka_temp_df.select(
    col("timestamp").alias("kafka_arrival_time"),
    from_json(col("value").cast("string"), schema_temp).alias("data")
).select(
    "kafka_arrival_time",
    "data.sensor_id",
    "data.temperature",
    col("data.timestamp").alias("sensor_timestamp"),
    "data.location"
).withColumn("event_timestamp", to_timestamp(col("sensor_timestamp")))

# 4. Función de Alertas 
def procesar_alertas(batch_df, batch_id):
    # Filtramos las temperaturas altas (> 25)
    alertas = batch_df.filter(col("temperature") > 25.0)
    
    if not alertas.isEmpty():
        print(f"\n --- ¡ALERTA DE ALTA TEMPERATURA! (Batch: {batch_id}) ---")
        print(f"Se han detectado {alertas.count()} lecturas críticas:")
        # Mostramos los detalles de la alerta
        alertas.select("sensor_id", "temperature", "location", "sensor_timestamp").show(truncate=False)
    else:
        # Esto indica que el sistema funciona pero las temperaturas son normales
        if not batch_df.isEmpty():
            print(f"Batch {batch_id}: {batch_df.count()} lecturas recibidas. Todo bajo control (<= 25°C).")

# 5. Inicio del Stream con Trigger de 5 segundos
query_alertas = parsed_temp_df.writeStream \
    .foreachBatch(procesar_alertas) \
    .trigger(processingTime="5 seconds") \
    .start()

print("Sistema de Alertas iniciado en SEGUNDO PLANO.")
print("Puedes seguir usando el cuaderno. Las alertas aparecerán aquí abajo.")

Sistema de Alertas iniciado en SEGUNDO PLANO.
Puedes seguir usando el cuaderno. Las alertas aparecerán aquí abajo.

 --- ¡ALERTA DE ALTA TEMPERATURA! (Batch: 1) ---
Se han detectado 9 lecturas críticas:
+---------+-----------+-----------+-------------------------+
|sensor_id|temperature|location   |sensor_timestamp         |
+---------+-----------+-----------+-------------------------+
|TEMP_01  |139.19     |Warehouse_A|2025-02-22T22:30:00+00:00|
|TEMP_01  |139.18     |Warehouse_A|2025-02-22T22:40:00+00:00|
|TEMP_01  |139.48     |Warehouse_A|2025-02-22T22:50:00+00:00|
|TEMP_01  |139.48     |Warehouse_A|2025-02-22T23:00:00+00:00|
|TEMP_01  |139.61     |Warehouse_A|2025-02-22T23:10:00+00:00|
|TEMP_01  |139.79     |Warehouse_A|2025-02-22T23:20:00+00:00|
|TEMP_01  |139.66     |Warehouse_A|2025-02-22T23:30:00+00:00|
|TEMP_01  |139.96     |Warehouse_A|2025-02-22T23:40:00+00:00|
|TEMP_01  |140.04     |Warehouse_A|2025-02-22T23:50:00+00:00|
+---------+-----------+-----------+------------------

In [6]:
# Detener TODOS los streams activos de golpe
for query in spark.streams.active:
    print(f"Deteniendo query: {query.name} (ID: {query.id})")
    query.stop()

print("Todos los procesos detenidos.")

Deteniendo query: None (ID: 349ff41f-2aff-45a1-868e-52f0c86ae161)
Todos los procesos detenidos.
